# Atmospheric Correction

In remote sensing we often need to calculate surface reflectance (ρ) from radiance (L) measure by a given sensor

### ρ = π(L -L<sub>p</sub>) / τ(E<sub>dir</sub> + E<sub>dif</sub>)

where

* ρ = surface reflectance
* L = at-sensor radiance
* L<sub>p</sub> = path radiance
* τ = transmissivity (from surface to satellite)
* E<sub>dir</sub> = direct solar irradiance 
* E<sub>dif</sub> = diffuse solar irradiance
* π = 3.1415



Let's say a satellite sensor measures a radiance of 120

In [2]:
L = 120

There are 4 unknowns remaining and they on i) atmospheric conditions and ii) Earth-Sun-Satellite geometry

**Atmospheric Conditions** <br />
let's say we have measured values of:

In [3]:
H2O = 1    # water vapour (g cm-2)
O3 = 0.4   # ozone (atm-cm)
AOT = 0.3  # aerosol optical thickness

**Earth-Sun-Satellite Geometry** <br />
and some additional measurements:

In [4]:
alt = 0      # target altitude (km)
solar_z = 20 # solar zenith angle (degrees)
view_z = 0   # view zentith angle (degrees)
doy = 4      # Earth-Sun distance (Astronomical Units)

**Potential data sources**

* Water vapour: [NCEP/NCAR](http://journals.ametsoc.org/doi/abs/10.1175/1520-0477%281996%29077%3C0437%3ATNYRP%3E2.0.CO%3B2)
* Ozone: [TOMS/OMI](http://ozoneaq.gsfc.nasa.gov/missions). 
* Aerosol optical thickness: [MODIS Aerosol Product](http://modis-atmos.gsfc.nasa.gov/MOD04_L2/index.html) or in-scene techniques
* Geometry and day-of-year: satellite image metadata


**6S emulator**

In [14]:
# loading dependencies
import xarray as xr
import rioxarray as rxr
import numpy as np
from Py6S import *
import datetime
import math
import os
import sys
import atmos
from datetime import datetime, timedelta
import act
import ee
import os
import sys
sys.path.append(os.path.join(os.path.dirname(os.getcwd()),'bin'))
from interpolated_LUTs import Interpolated_LUTs

The 6S emulator is **100x** faster than the radiative transfer code. This speed increased is acheived by using interpolated look-up tables (iLUTs). This trades set-up time for execution time.

In [6]:
# instantiate interpolated look up table class
iLUTs = Interpolated_LUTs('COPERNICUS/S2') # i.e. Sentinel 2

files directory not found, will create at:
/home/jovyan/6S_emulator/files
LUT directory created:
/home/jovyan/6S_emulator/files/LUTs/S2A_MSI/Continental/view_zenith_0
iLUT directory created:
/home/jovyan/6S_emulator/files/iLUTs/S2A_MSI/Continental/view_zenith_0


In [7]:
# download look-up tables
iLUTs.download_LUTs()

Extracting zip file..
Done: LUT files available locally


In [8]:
# interpolate look-up tables
iLUTs.interpolate_LUTs()

running n-dimensional interpolation may take a few minutes...
Interpolating: S2A_MSI_01.lut
Interpolation took 22.79 (secs) = 
Interpolating: S2A_MSI_02.lut
Interpolation took 22.92 (secs) = 
Interpolating: S2A_MSI_03.lut
Interpolation took 23.00 (secs) = 
Interpolating: S2A_MSI_04.lut
Interpolation took 23.62 (secs) = 
Interpolating: S2A_MSI_05.lut
Interpolation took 22.65 (secs) = 
Interpolating: S2A_MSI_06.lut
Interpolation took 22.68 (secs) = 
Interpolating: S2A_MSI_07.lut
Interpolation took 22.53 (secs) = 
Interpolating: S2A_MSI_08.lut
Interpolation took 22.40 (secs) = 
Interpolating: S2A_MSI_09.lut
Interpolation took 22.39 (secs) = 
Interpolating: S2A_MSI_10.lut
Interpolation took 22.47 (secs) = 
Interpolating: S2A_MSI_11.lut
Interpolation took 22.98 (secs) = 
Interpolating: S2A_MSI_12.lut
Interpolation took 23.14 (secs) = 
Interpolating: S2A_MSI_13.lut
Interpolation took 23.19 (secs) = 


If you are running this notebook in a docker container then you can save these interpolated look-up tables (and your Earth Engine authentication) for later using a [docker commit](https://github.com/samsammurphy/6S_emulator/wiki/docker-commits). This will save them to memory so that you only have to do the set-up once.

In [1]:
iLUTs_all_wavebands = iLUTs.get()
iLUTs_all_wavebands

NameError: name 'iLUTs' is not defined

In [15]:
# for example let's look at band 1
iLUT_B1 = iLUTs_all_wavebands['B1']

We can get atmospheric correction coefficients (a, b) for *perihelion*:

In [16]:
a, b = iLUT_B1(solar_z,H2O,O3,AOT,alt)
a,b

(64.7571, 393.9648244144661)

and corrected for Earth's [elliptical orbit](https://github.com/samsammurphy/6S_LUT/wiki/Elliptical-Orbit-Correction):

In [19]:
import math

elliptical_orbit_correction = 0.03275104*math.cos(doy/59.66638337) + 0.96804905
a *= elliptical_orbit_correction
b *= elliptical_orbit_correction
a,b

(64.85122900782251, 394.5374800466532)

to get the correction coefficients we need to calculate surface reflectance.

In [20]:
ρ = (L-a)/b

print('Surface Reflectance = {:.3f}'.format(ρ))

Surface Reflectance = 0.140
